In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

In [ ]:
dff = pd.read_csv("Data/Raw/311_data.csv")
dff

/var/folders/w0/4r14_1sj0j7fl6nz7v1nwgkr0000gp/T/ipykernel_36148/3499492199.py:1: DtypeWarning: Columns (34) have mixed types. Specify dtype option on import or set low_memory=False.
  dff = pd.read_csv("Data/Raw/311_data.csv")


In [ ]:
dff

In [15]:
# drop corrupted rows at the end
df = dff.iloc[:2872754,:]
df.drop(["Unique Key", "Closed Date", "Agency", "Agency Name","Incident Address",'X Coordinate (State Plane)',
       'Y Coordinate (State Plane)', 'Open Data Channel Type', "Location Type",
       'Park Facility Name', 'Park Borough', 'Vehicle Type', "Status",
       'Taxi Company Borough', 'Taxi Pick Up Location', 'Bridge Highway Name',
       'Bridge Highway Direction', 'Road Ramp', 'Bridge Highway Segment','Due Date',
       'Resolution Description', 'Resolution Action Updated Date',
       'Community Board', 'BBL','Incident Address', 'Street Name', 'Cross Street 1', 'Cross Street 2',
       'Intersection Street 1', 'Intersection Street 2', 'Address Type',
       'City', 'Landmark', 'Facility Type'], axis=1, inplace=True)

# drop rows with no location associated
df.dropna(subset='Latitude',inplace=True)
df.reset_index(drop=True,inplace=True)

/var/folders/w0/4r14_1sj0j7fl6nz7v1nwgkr0000gp/T/ipykernel_21542/4152900371.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(["Unique Key", "Closed Date", "Agency", "Agency Name","Incident Address",'X Coordinate (State Plane)',
/var/folders/w0/4r14_1sj0j7fl6nz7v1nwgkr0000gp/T/ipykernel_21542/4152900371.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.dropna(subset='Latitude',inplace=True)


In [16]:
# get census tract to coordinate conversion

# Load the census tract shapefile
tracts = gpd.read_file('Data/Raw/tracts2020_shapefile/nyct2020.shp')
tracts = tracts.to_crs(epsg = 4326)

coords_df = df[["Latitude","Longitude"]].drop_duplicates().reset_index(drop=True)
coords = [tuple(record) for record in coords_df.to_records(index=False)] #[(40.735324, -73.998004), (40.715595,	-73.987030)]

# Create a GeoDataFrame from the coordinates
geometry = [Point(lon, lat) for lat, lon in coords]
geo_df = gpd.GeoDataFrame(geometry=geometry, crs="EPSG:4326")
geo_df["Latitude"] = coords_df["Latitude"]
geo_df["Longitude"] = coords_df["Longitude"]

# Perform a spatial join to match points to census tracts
ct_lookup = gpd.sjoin(geo_df, tracts, how='left', predicate="within")[["Latitude", "Longitude", "GEOID"]]

# Display results
ct_lookup

,Latitude,Longitude,GEOID
0,40.812650,-73.939490,36061021200
1,40.626782,-74.005995,36047021000
2,40.868979,-73.917383,36061030300
3,40.685954,-73.972398,36047018100
4,40.799986,-73.947057,36061018600
...,...,...,...
613823,40.731262,-73.874097,36081047500
613824,40.622026,-74.023984,36047014200
613825,40.665249,-73.915415,36047090200
613826,40.622834,-74.017752,36047020600


In [44]:
# get service request location from dictionary lookup
df_ct = df.merge(ct_lookup, left_on=["Latitude", "Longitude"], right_on=["Latitude", "Longitude"])
df_ct.rename(columns={"GEOID":"Tract"}, inplace=True)

# remove first two digits from census tract number
df_ct['Tract'] = df_ct['Tract'].apply(lambda x: str(x)[3:])
df_ct.drop(['Latitude','Longitude', 'Location', 'Incident Zip'], axis=1, inplace=True)
df_ct

,Created Date,Complaint Type,Descriptor,Borough,Tract
0,06/28/2019 12:00:00 PM,Derelict Vehicles,Derelict Vehicles,MANHATTAN,61021200
1,06/27/2019 12:00:00 PM,Derelict Vehicles,Derelict Vehicles,BROOKLYN,47021000
2,06/28/2019 12:00:00 PM,Derelict Vehicles,Derelict Vehicles,MANHATTAN,61030300
3,06/27/2019 12:00:00 PM,Derelict Vehicles,Derelict Vehicles,BROOKLYN,47018100
4,06/28/2019 12:00:00 PM,Derelict Vehicles,Derelict Vehicles,MANHATTAN,61018600
...,...,...,...,...,...
2788826,08/05/2020 07:24:23 PM,Noise - Street/Sidewalk,Loud Music/Party,MANHATTAN,61013300
2788827,08/05/2020 08:54:39 PM,Damaged Tree,Branch Cracked and Will Fall,QUEENS,81049700
2788828,08/05/2020 12:59:23 PM,Damaged Tree,Branch or Limb Has Fallen Down,MANHATTAN,61018400
2788829,08/05/2020 01:22:24 PM,Damaged Tree,Branch Cracked and Will Fall,STATEN ISLAND,85020803


In [45]:
### Condense Complaint Type Categories

conditions = [df_ct["Complaint Type"].str.contains("oise|Illegal Fireworks"),
             (df_ct["Complaint Type"].str.contains("UNSANITARY CONDITION|Sanitation Condition|Dirty Conditions|Unsanitary Condition")|df_ct["Descriptor"].str.contains("Dirty|Debris|Odor|MOLD|Pigeon Waste")),
             (df_ct["Complaint Type"].str.contains("Water System|Sewer|PLUMBING|WATER LEAK")|df_ct["Descriptor"].str.contains("WATER")),
             (df_ct["Complaint Type"].str.contains("Illegal Parking|Blocked Driveway|Vehicle")|df_ct["Descriptor"].str.contains("Vehicle|Parking|Blocked|Blocking|License")),
             (df_ct["Complaint Type"].str.contains("Request Large Bulky Item Collection|Missed Collection|Electronics Waste Appointment")|df_ct["Descriptor"].str.contains("Collection|waste|All Materials|Garbage|Recycling")),
             (df_ct["Complaint Type"].str.contains("Damaged Tree|General Construction/Plumbing|Condition|Broken|Street Sign - Damaged")|df_ct["Descriptor"].str.contains("Broken|Pothole|Branch|Cave-in|BROKEN|Street Light Out|Fallen")),
             df_ct["Complaint Type"].str.contains("Homeless"),
             (df_ct["Complaint Type"].str.contains("Rodent")|df_ct["Descriptor"].str.contains("Rodent|PESTS|Rat|Mouse"))]

choices = ["Noise_311","Sanitation_311","Water_311","Vehicle_311","Trash_311","Damage_311","Homeless_311","Pests_311"]

df_ct["Complaint Type"] = np.select(conditions, choices, "Other_311")


In [46]:
### Convert date column to datetime and bucket into years

df_ct["Created Date"] = pd.to_datetime( df_ct["Created Date"])
df_ct["Created Date"] = df_ct["Created Date"].apply(lambda x: x.year)
df_ct.rename(columns={'Created Date':'Year'},inplace=True)
df_ct.sample(5)

,Year,Complaint Type,Descriptor,Borough,Tract
1671851,2020,Noise_311,Banging/Pounding,QUEENS,81094203
739687,2019,Vehicle_311,Partial Access,BROOKLYN,47072400
2538121,2020,Noise_311,Banging/Pounding,STATEN ISLAND,85027301
2615596,2020,Water_311,Fire Hydrant Emergency (FHE),MANHATTAN,61029500
1861333,2020,Other_311,POWER OUTAGE,BRONX,05035900


In [48]:
### Reshape Data

grouped = df_ct.groupby(["Tract", "Year","Complaint Type"])["Descriptor"].count().rename("Count").reset_index()
piv = grouped.pivot_table(columns="Complaint Type", values="Count", index=["Tract","Year"])
piv.sample(5)


,Complaint Type,Damage_311,Homeless_311,Noise_311,Other_311,Pests_311,Sanitation_311,Trash_311,Vehicle_311,Water_311
Tract,Year,,,,,,,,,
05039200,2020,24.0,NaN,43.0,81.0,1.0,17.0,31.0,65.0,12.0
85024800,2020,155.0,0.0,105.0,249.0,3.0,51.0,244.0,112.0,42.0
81040200,2019,37.0,NaN,36.0,32.0,3.0,38.0,56.0,42.0,12.0
47039400,2020,96.0,NaN,20.0,102.0,NaN,12.0,84.0,61.0,12.0
05011900,2019,55.0,0.0,102.0,301.0,16.0,41.0,13.0,27.0,56.0


In [49]:
# piv.to_parquet('Data/Cleaned/311_requests.parquet')